In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.retrieval_model import RetrievalEditorModel
from coeditor.api import EditPredictionService, BasicQueryEditEncoder, BatchArgs

model_path = get_model_dir(True) / "coeditor-large-query-basic"
model = RetrievalEditorModel.load(model_path)
model.to("cuda:1")
batch_args = BatchArgs.service_default()
service = EditPredictionService(
    proj_root(),
    model,
    batch_args=batch_args,
    encoder=BasicQueryEditEncoder(
        max_ref_tks=batch_args.max_ref_tks,
        max_query_tks=batch_args.max_query_tks,
        max_output_tks=batch_args.max_output_tks,
    ),
)


In [12]:
service.suggest_edit(Path("src/coeditor/encoders.py"), 138)
# service.suggest_edit(Path("src/coeditor/api.py"), 277)
# service.suggest_edit(Path("src/spot/utils.py"), (218, 0))


num batches: 1
Batch stats: {'mean': '1.0', 'median': '1.0', 'min': '1.0', 'max': '1.0'}
========== Predicted code change ==========
* Modified: 
    ... 4 lines omitted ...
            def encode_pedit(
                self,
                pedit: ProjectEdit,
                include_additions: bool = False,
                queries: Iterable[Modified[PythonFunction]] | None = None,
            ) -> Iterable[BasicTkQueryEdit]:
    -           ctx_enc = CtxEncoder(pedit, self.collapse_unchanged, indent_in_class=False)
    +           ctx_enc = CtxEncoder(pedit, self.collapse_unchanged)
                moved = find_moved(pedit.all_elem_changes())
                moved_paths = {a for a, b in moved} | {b for a, b in moved}
                after_to_mf = {
                    b: mf
                    for (a, b), change in moved.items()
                    if (mf := to_modified_function(change))
                }
    +           module_stubs = None
    +           if self.add_stubs:
    +   

In [ ]:
service.tlogger.as_dataframe()


,name,count,avg_time,total_time
3,run model,1,2.703628,2.703628
1,construct project edit,1,0.398109,0.398109
0,get target element,1,0.123962,0.123962
2,encode edits,1,0.104199,0.104199
